In [29]:
# Import necessary packages for notebook
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, TimestampType
import warnings

In [13]:
spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

In [14]:
raw_games = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:29092") \
        .option("subscribe", "games") \
        .option("startingOffsets", "earliest") \
        .option("endingOffsets", "latest") \
        .load()

In [15]:
games = raw_games.select(raw_games.value.cast('string'))

# final_schema = StructType([StructField('GameKey', StringType(), True),StructField('AwayTeam', StringType(), True),StructField('HomeTeam', StringType(), True)])


games_df = games.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [48]:
final_schema = StructType([StructField('GameKey', StringType(), True),
                     StructField('Week', StringType(), True),
                     StructField('AwayTeam', StringType(), True),
                     StructField('AwayScore', IntegerType(), True),
                     StructField('HomeTeam', StringType(), True),
                     StructField('HomeScore', IntegerType(), True),
                     StructField('PointSpread', FloatType(), True),
                     StructField('OverUnder', FloatType(), True),
                     StructField('AwayTeamMoneyLine', IntegerType(), True),
                     StructField('HomeTeamMoneyLine', IntegerType(), True)])

In [49]:
games_df = games.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [50]:
games_df.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110804|   8|     MIA|       17|     BUF|       41|      -22.9|     76.4|             1032|            -1592|
|202110806|   8|      SF|       52|     CHI|       35|        9.9|     63.0|             -325|              271|
|202110808|   8|     PIT|       24|     CLE|       16|       -8.7|     67.8|              282|             -344|
|202110811|   8|     PHI|       69|     DET|       15|        8.7|     76.4|             -274|              232|
|202110814|   8|     TEN|       54|     IND|       49|       -4.7|     80.4|              206|  

In [17]:
games_df \
        .write \
        .parquet("/tmp/games")

In [51]:
# Read from parquet to ensure that it worked
read_table = spark.read.parquet('/tmp/games')
read_table.show()

+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|  GameKey|Week|AwayTeam|AwayScore|HomeTeam|HomeScore|PointSpread|OverUnder|AwayTeamMoneyLine|HomeTeamMoneyLine|
+---------+----+--------+---------+--------+---------+-----------+---------+-----------------+-----------------+
|202110802|   8|     CAR|       30|     ATL|       20|       -2.4|     74.1|              169|             -202|
|202110804|   8|     MIA|       17|     BUF|       41|      -22.9|     76.4|             1032|            -1592|
|202110806|   8|      SF|       52|     CHI|       35|        9.9|     63.0|             -325|              271|
|202110808|   8|     PIT|       24|     CLE|       16|       -8.7|     67.8|              282|             -344|
|202110811|   8|     PHI|       69|     DET|       15|        8.7|     76.4|             -274|              232|
|202110814|   8|     TEN|       54|     IND|       49|       -4.7|     80.4|              206|  